In [4]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import json

# Set the User-Agent header to mimic a browser
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

# List of profile URLs to scrape
profile_urls = [
    # Add profile URLs as needed
]

# Lists to store data
Statut_Depuis_2018 = []  # List to store status since 2018
Statut_global = []      # List to store global status
ArticleListe = []       # List to store article information

# Function to extract data from the table of statistics
def gettableofsat(soup):
    table_of_sat = soup.find(id='gsc_rsb_st')
    body_of_table = table_of_sat.contents[1]
    hs_indeces = body_of_table.contents[1].find_all("td", {'class': 'gsc_rsb_std'})
    Citations = body_of_table.contents[0].find_all("td", {'class': 'gsc_rsb_std'})
    h_indice_I10 = body_of_table.contents[2].find_all("td", {'class': 'gsc_rsb_std'})

    Statut_Depuis_2018.append({"H indice": hs_indeces[0].text.strip(), "Citations": Citations[0].text.strip(),
                               "H indice_I10": hs_indeces[0].text.strip()})
    Statut_global.append({"H indice": hs_indeces[1].text.strip(), "Citations": Citations[1].text.strip(),
                          "H indice_I10": h_indice_I10[1].text.strip()})

# Function to get the name of the profile
def getNameProfil(soup):
    NomProfil = soup.find(id='gsc_prf_in').text.strip()
    return NomProfil

# Function to get the associated university or institution
def getUnevercity(soup):
    assc_or_univer = soup.find("div", {'class': 'gsc_prf_il'})
    Assc_unevercity = assc_or_univer.get_text(strip=True)
    return Assc_unevercity

# Function to get the specialty of the profile
def spicialty(soup):
    spicialty = soup.find(id="gsc_prf_int")
    Specialty = spicialty.get_text(strip=True)
    return Specialty

# Function to extract information about all articles
def getArticle(soup):
    articles = []
    for article in soup.find_all("tr", {'class': 'gsc_a_tr'}):
        article_info = {
            "Nom Article": article.find("a", {'class': 'gsc_a_at'}).text.strip(),
            "description": article.find("div", {'class': 'gs_gray'}).text.strip(),
            "auteur": article.find("div", {'class': 'gs_gray'}).find_next("div", {'class': 'gs_gray'}).text.strip(),
            "Annee": article.find("span", {'class': 'gsc_a_h gsc_a_hc gs_ibl'}).text.strip(),
        }
        articles.append(article_info)
    return articles

# Function to scrape profile information
def scrape_profile(url):
    page = requests.get(url, headers=headers)
    src = page.content
    soup = BeautifulSoup(src, "lxml")

    NomProfil = getNameProfil(soup)
    Assc_unevercity = getUnevercity(soup)
    Specialty = spicialty(soup)
    gettableofsat(soup)
    ArticleListe = getArticle(soup)

    return {
        "Nom et Prénom": NomProfil,
        "ASS - Unevercity": Assc_unevercity,
        "Spciality": Specialty,
        "Status Depuis 2018": Statut_Depuis_2018,
        "Status Golobal": Statut_global,
        "Article": ArticleListe
    }

# Function to save profile data to a JSON file
def save_to_json(profiles, filename='profiles.json'):
    with open(filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(profiles, jsonfile, ensure_ascii=False, indent=2)

# Main function to iterate through profile URLs and scrape data
def main():
    all_profiles = []

    for profile_url in profile_urls:
        profile_data = scrape_profile(profile_url)
        all_profiles.append(profile_data)

    save_to_json(all_profiles)

# Check if the script is executed as the main program
if __name__ == "__main__":
    main()
